In [1]:
# import the libraries
from pathlib import Path
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

# Processing data

In [2]:
# extracting paths for data and results
data_path = Path().cwd().parent/'Data'
result_path = Path().cwd().parent/'Results'

In [3]:
# load data
train_df = pd.read_csv(data_path/'train.csv')
test_df = pd.read_csv(data_path/'test.csv')

In [4]:
# get unmarked data
unmarked_train = train_df.query("target==-1")
unmarked_train.drop('target', axis=1, inplace=True)
unmarked_train.head(2)

/tmp/ipykernel_108117/3347485245.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unmarked_train.drop('target', axis=1, inplace=True)


,index,name,description
0,324865089,продавец кассир,<strong>Обязанности:</strong> <ul> <li>работа ...
1,169467135,продавец мила (шевченко 17),<p><strong>Магазин МИЛА по адресу б-р Шевченко...


In [5]:
# get rid of unmarked data
train_df_marked = train_df.query("target!=-1")
train_df_marked.head(2)

,index,name,description,target
2,169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223
5,169293782,продавец консультант yota (тц галерея),<p>За любыми достижениями нашей компании в пер...,5223


In [6]:
test_df.head(2)

,index,name,description
0,26461447,"персональный водитель сервиса ""wheely""",<p><strong>В </strong>связи с расширением авто...
1,26464220,менеджер по автоперевозкам,<strong>Обязанности:</strong> <ul> <li>Поиск п...


In [7]:
# load and merge json data
def json_data_getter(json_dic):
    result = []
    for json_file in json_dic:
        if json_file.get('Обязанности', np.nan) is not np.nan:
            result.append(' '.join(json_file['Обязанности']))
        else:
            result.append(np.nan)
    return result

def json_processor(path):
    json_df = pd.read_json(path)
    json_df['useful_data'] = json_data_getter(json_df['Content'])
    json_df.drop(['Content', 'Position'], axis=1, inplace=True)
    return json_df

vd1 = json_processor(data_path/'vacancy_descriptions'/'1_parsed.json')
vd2 = json_processor(data_path/'vacancy_descriptions'/'2_parsed.json')
vd3 = json_processor(data_path/'vacancy_descriptions'/'3_parsed.json')
vd4 = json_processor(data_path/'vacancy_descriptions'/'4_parsed.json')
vd5 = json_processor(data_path/'vacancy_descriptions'/'5_parsed.json')
vd = pd.concat([vd1, vd2, vd3, vd4, vd5], ignore_index=True)
vd.head(5)

,ID,useful_data
0,363107946,NaN
1,363125198,NaN
2,363144355,NaN
3,363340232,NaN
4,363383054,Организовывать и проводить просмотры/показы об...


In [8]:
# merge json data with train
train_df_marked_w_json = train_df_marked.merge(vd, how='left', left_on='index', right_on='ID')
train_df_marked_w_json.drop(['ID'], inplace=True, axis=1)
train_df_marked_w_json.head(2)

,index,name,description,target,useful_data
0,169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223,":принимать и выдавать заказы, расчет с гостями..."
1,169293782,продавец консультант yota (тц галерея),<p>За любыми достижениями нашей компании в пер...,5223,:Рассказывать людям о Yota Продавать устройств...


In [9]:
len(train_df_marked_w_json[train_df_marked_w_json['useful_data'].isna()])

2311

In [10]:
# transform train data
def simple_transformer(sample_description, stop_words = stopwords.words('russian'), morph = MorphAnalyzer()):
    if sample_description is np.nan:
        return np.nan
    patt1 = re.compile(r'[а-я\s]')
    sample_description_v2 = ''.join(re.findall(patt1, sample_description.lower())).replace('\n', '')
    sample_description_v3 = ' '.join([word for word in sample_description_v2.split(' ') if word not in stop_words])
    inf_forms = ' '.join([morph.parse(word)[0].normal_form for word in sample_description_v3.split()])
    return inf_forms

train_df_marked_w_json.loc[:,'redacted description'] = train_df_marked_w_json.apply(lambda row: simple_transformer(row['useful_data']), axis=1)
train_df_marked_w_json['redacted name'] = train_df_marked_w_json.apply(lambda row: simple_transformer(row['name']), axis=1)
train_df_marked_w_json['full description'] = train_df_marked_w_json['redacted name'] + ' ' + train_df_marked_w_json['redacted description']
train_df_marked_w_json.drop(['useful_data', 'description', 'index', 'redacted description', 'name'], axis=1, inplace=True)
train_df_marked_w_json.head(2)

,target,redacted name,full description
0,5223,кассир пиццерия г витебск,кассир пиццерия г витебск принимать выдавать з...
1,5223,продавец консультант тц галерея,продавец консультант тц галерея рассказывать ч...


In [11]:
len(train_df_marked_w_json[train_df_marked_w_json['full description'].isna()])

2311

In [12]:
# fill nan description
train_df_marked_w_json['full description'].fillna(train_df_marked_w_json['redacted name'], inplace=True)
len(train_df_marked_w_json[train_df_marked_w_json['full description'].isna()])

0

In [13]:
train_df_marked_w_json.drop('redacted name', axis=1, inplace=True)
train_df_marked_w_json

,target,full description
0,5223,кассир пиццерия г витебск принимать выдавать з...
1,5223,продавец консультант тц галерея рассказывать ч...
2,7212,электросварщик накс ск п г норильск ручной дуг...
3,5223,кассир правило приём проведение расчётный конв...
4,6121,оператор машинный доение выполнять вид работа ...
...,...,...
15645,3341,офис менеджер
15646,3341,офис менеджер приём отправка входящий исходящи...
15647,3341,офис менеджер встречать направлять клиент вход...
15648,3341,офис менеджер приём регистрация входящий звоно...


In [14]:
# same procedure with unmarked data
unmarked_train_w_json = unmarked_train.merge(vd, how='left', left_on='index', right_on='ID')
unmarked_train_w_json.drop(['ID'], inplace=True, axis=1)
unmarked_train_w_json.head(2)

,index,name,description,useful_data
0,324865089,продавец кассир,<strong>Обязанности:</strong> <ul> <li>работа ...,работа с кассой : открытие / закрытие смены ве...
1,169467135,продавец мила (шевченко 17),<p><strong>Магазин МИЛА по адресу б-р Шевченко...,выкладка товара в соответствии с планограммой ...


In [15]:
len(unmarked_train_w_json[unmarked_train_w_json['useful_data'].isna()])

2120

In [16]:
unmarked_train_w_json.loc[:,'redacted description'] = unmarked_train_w_json.apply(lambda row: simple_transformer(row['useful_data']), axis=1)
unmarked_train_w_json['redacted name'] = unmarked_train_w_json.apply(lambda row: simple_transformer(row['name']), axis=1)
unmarked_train_w_json['full description'] = unmarked_train_w_json['redacted name'] + ' ' + unmarked_train_w_json['redacted description']
unmarked_train_w_json.drop(['useful_data', 'description', 'index', 'redacted description', 'name'], axis=1, inplace=True)
unmarked_train_w_json.head(2)

,redacted name,full description
0,продавец кассир,продавец кассир работа касса открытие закрытие...
1,продавец мила шевченко,продавец мила шевченко выкладка товар соответс...


In [17]:
len(unmarked_train_w_json[unmarked_train_w_json['full description'].isna()])

2120

In [18]:
# fill nan description
unmarked_train_w_json['full description'].fillna(unmarked_train_w_json['redacted name'], inplace=True)
len(unmarked_train_w_json[unmarked_train_w_json['full description'].isna()])

0

In [19]:
unmarked_train_w_json.drop('redacted name', axis=1, inplace=True)
unmarked_train_w_json.head(2)

,full description
0,продавец кассир работа касса открытие закрытие...
1,продавец мила шевченко выкладка товар соответс...


In [20]:
# merge json data with test
test_df_w_json = test_df.merge(vd, how='left', left_on='index', right_on='ID')
test_df_w_json.drop(['ID'], inplace=True, axis=1)
test_df_w_json.head(2)

,index,name,description,useful_data
0,26461447,"персональный водитель сервиса ""wheely""",<p><strong>В </strong>связи с расширением авто...,Выполнение поступающих заявок через приложение...
1,26464220,менеджер по автоперевозкам,<strong>Обязанности:</strong> <ul> <li>Поиск п...,Поиск поставщиков Ведение перевозок от начало ...


In [21]:
len(test_df_w_json)

1090

In [22]:
# number of nan values in the test df
sum(test_df_w_json['useful_data'].isna())

182

In [23]:
# process test data
test_df_w_json['redacted name'] = test_df_w_json.apply(lambda row: simple_transformer(row['name']) , axis=1)
test_df_w_json['redacted description'] = test_df_w_json.apply(lambda row: simple_transformer(row['useful_data']), axis=1)
test_df_w_json['full description'] = test_df_w_json['redacted name'] + ' ' + test_df_w_json['redacted description']
test_df_w_json['full description'].fillna(test_df_w_json['redacted name'], inplace=True)
test_df_w_json.drop(['redacted name', 'redacted description', 'name', 'useful_data'], axis=1, inplace=True)
test_df_w_json.head(2)

,index,description,full description
0,26461447,<p><strong>В </strong>связи с расширением авто...,персональный водитель сервис выполнение поступ...
1,26464220,<strong>Обязанности:</strong> <ul> <li>Поиск п...,менеджер автоперевозка поиск поставщик ведение...


In [24]:
len(test_df_w_json[test_df_w_json['full description'].isna()])

0

In [25]:
test_df_w_json.drop('description', axis=1, inplace=True)
test_df_w_json.head(2)

,index,full description
0,26461447,персональный водитель сервис выполнение поступ...
1,26464220,менеджер автоперевозка поиск поставщик ведение...


# Machine learning

### Training

In [26]:
train_df_marked_w_json

,target,full description
0,5223,кассир пиццерия г витебск принимать выдавать з...
1,5223,продавец консультант тц галерея рассказывать ч...
2,7212,электросварщик накс ск п г норильск ручной дуг...
3,5223,кассир правило приём проведение расчётный конв...
4,6121,оператор машинный доение выполнять вид работа ...
...,...,...
15645,3341,офис менеджер
15646,3341,офис менеджер приём отправка входящий исходящи...
15647,3341,офис менеджер встречать направлять клиент вход...
15648,3341,офис менеджер приём регистрация входящий звоно...


In [27]:
X = train_df_marked_w_json['full description']
Y = train_df_marked_w_json['target']

In [29]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
vectorizer = TfidfVectorizer()
tf_train = vectorizer.fit_transform(x_train)
tf_test = vectorizer.transform(x_test)
pas_ag = PassiveAggressiveClassifier()
pas_ag.fit(tf_train, y_train)

PassiveAggressiveClassifier()

### Testing

In [30]:
pas_ag.score(tf_test, y_test)

0.9383386581469648

### Predicting for unmarked data

In [31]:
tf_predict = vectorizer.transform(unmarked_train_w_json['full description'])
code_predict = pas_ag.predict(tf_predict)
code_predict

array([5223, 5223, 3123, ..., 3341, 9622, 6210])

In [32]:
# marking the unmarked data
unmarked_train_w_json['target'] = code_predict
unmarked_train_w_json.head(2)

,full description,target
0,продавец кассир работа касса открытие закрытие...,5223
1,продавец мила шевченко выкладка товар соответс...,5223


In [33]:
# concatting training
ultimate_training = pd.concat([train_df_marked_w_json, unmarked_train_w_json])
ultimate_training.head(5)

,target,full description
0,5223,кассир пиццерия г витебск принимать выдавать з...
1,5223,продавец консультант тц галерея рассказывать ч...
2,7212,электросварщик накс ск п г норильск ручной дуг...
3,5223,кассир правило приём проведение расчётный конв...
4,6121,оператор машинный доение выполнять вид работа ...


In [34]:
len(ultimate_training)

30000

### Fitting another model

In [35]:
# performing train test split
X = ultimate_training['full description']
Y = ultimate_training['target']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

Just passive aggressive classifier

In [62]:
pass_agg1 = PassiveAggressiveClassifier(max_iter=5000)
tf_train1 = vectorizer.transform(x_train)
pass_agg1.fit(tf_train1, y_train)

PassiveAggressiveClassifier(max_iter=5000)

In [63]:
tf_test1 = vectorizer.transform(x_test)
pass_agg1.score(tf_test1, y_test)

0.9626666666666667

In [64]:
tf_predict1 = vectorizer.transform(test_df_w_json['full description'])
prof_predict = pass_agg1.predict(tf_predict1)
prof_predict

array([8322, 3323, 5223, ..., 5223, 2269, 4323])

My pipeline

In [36]:
pass_agg_pip = Pipeline([
    ('vect', TfidfVectorizer()),
    ('pag', PassiveAggressiveClassifier(max_iter=10000))
])

In [37]:
pass_agg_pip.fit(x_train, y_train)

Pipeline(steps=[('vect', TfidfVectorizer()),
                ('pag', PassiveAggressiveClassifier(max_iter=10000))])

In [38]:
pass_agg_pip.score(x_test, y_test)

0.9603333333333334

In [39]:
prof_predict = pass_agg_pip.predict(test_df_w_json['full description'])
prof_predict

array([8322, 3323, 5223, ..., 5223, 3331, 4323])

Buddy's pipeline

In [48]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='modified_huber', penalty='l2',
                         alpha=1e-4, random_state=42,
                         max_iter=5, tol=None)),
])

In [49]:
text_clf = text_clf.fit(x_train, y_train)
text_clf.score(x_test, y_test)

0.9666666666666667

In [50]:
prof_predict = text_clf.predict(test_df_w_json['full description'])
prof_predict

array([8322, 3323, 5223, ..., 5223, 8343, 4323])

# Запись результата

In [40]:
results_to_write1 = test_df_w_json['index']
prof_predict = pd.Series(prof_predict)
prof_predict.name = 'target'
results_to_write1 = pd.concat([results_to_write1, prof_predict], axis=1)
results_to_write1

,index,target
0,26461447,8322
1,26464220,3323
2,26467473,5223
3,26468989,2411
4,26471705,4222
...,...,...
1085,123809156,9622
1086,104700805,4323
1087,114563177,5223
1088,354950498,3331


In [41]:
results_to_write1.to_csv(result_path/'combobo_v4.csv', index=False)